In [1]:
#!pip install openai python-dotenv
#!pip install unidecode

# Generic
import pandas as pd

# For plot_output (part 1)
import matplotlib
matplotlib.use('agg')

# To get started
import openai
import os

# For preprocess_data
import glob
from datetime import datetime
from unidecode import unidecode

# For create_batches
from transformers import GPT2Tokenizer

# For get_ratings
import re
import json

# For plot_output
import numpy as np
import matplotlib.pyplot as plt

# For panel
import panel as pn
import seaborn as sns
import random

In [2]:
result_string=[]
categories_loop=[]
data=[]
batches=[]
categories=[]
batch=[]
prompt=[]
final_df=[]

In [3]:
# Get started with OpenAI API

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
# Function to generate LLM response based on prompt

def get_completion(prompt, model="gpt-3.5-turbo"): 
    
    '''Simple function to open a chat conversation with a specified model, based on a given prompt, 
    returns the model's answer'''
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    
    print(f'{response["usage"]["prompt_tokens"]} prompt tokens used.')
    
    return response.choices[0].message["content"]

In [5]:
# Function to preprocess data

def preprocess_data(data):
    
    '''Function to preprocess raw input from outscraper file to result_string to feed into LLM
    Includes transformation to unicode characters
    Returns both full string to feed into LLM and full dataframe''' 
    
    # Parse date information
    data['review_date'] = pd.to_datetime(data['review_datetime_utc']).dt.strftime('%d-%m-%Y')
    data['owner_answer_date'] = pd.to_datetime(data['owner_answer_timestamp_datetime_utc']).dt.strftime('%d-%m-%Y')
    data['review_date'] = pd.to_datetime(data['review_date'], dayfirst=True)
    data['owner_answer_date'] = pd.to_datetime(data['owner_answer_date'], dayfirst=True)
    data['review_year'] = data['review_date'].dt.year
    data['answer_time'] = (data['owner_answer_date'] - data['review_date']).dt.days
    
    data['review_text_unicode'] = data['review_text'].apply(unidecode)
    
    # Only keep required columns
    columns_to_select = ['name', 'review_text_unicode', 'review_rating', 'review_likes', 'review_year',
                         'review_date', 'owner_answer', 'owner_answer_date', 'answer_time']
    df = data[columns_to_select].drop_duplicates()
   
    # Directly create reviews_list without using an intermediate string
    reviews_list = [f'<ID={idx}; review={unidecode(row["review_text"])}>' for idx, row in data.iterrows()]
    
    # Output string and df
    return reviews_list, df, data

In [6]:
# Function to create JSON output format for LLM quary

def review_categories(categories):
    
    '''Simple function to create a JSON template based on a list of review categories'''
    
    categories_loop = []

    for i in range(len(categories)):
        temp = f'''
        "Id": "ID", "category": "{categories[i]}","assessment": <your findings in English>,"score": <your score>
        '''
        categories_loop.append(temp)
    
    return categories_loop

In [7]:
# Function to create prompt

def create_prompt(result_string=result_string, categories_loop=categories_loop):
    
    '''Function intended to easily change prompt and create prompt without result string (call create_prompt(""))'''
    
    prompt = f"""
    You are a skilled analyzer of google maps reviews, multilingual with an eye for nuance.\

    Your output must be in valid JSON. Do not output anything other than the JSON.\

    You will analyze a list of reviews, where each review is surrounded by tags and is structured like \
    <ID=number; review=review>, based on a number of categories. For each category you will include\
    your findings in English and an integer score between 1 (very negative) and 5 (very positive).\
    If you do not have enough information, give a score of 0. Your assessment should not be a translation\
    of the review, but a different, succint summary of no more than 50 characters for each of the categories.\

    The categories are listed in the JSON template below.\

    Your output will be nothing other than a valid JSON, structured as follows:\
    '''{categories_loop}'''

    Surround your JSON output with <result></result> tags.

    Review text: '''{result_string}'''
    """
    
    return prompt

In [18]:
# Function to create batches of reviews to stay below token limit

def create_batches(data=data, categories=categories, total_token_limit=4000):

    '''Given the token limit for all LLMs, create batches of input and estimated output to stay below limit'''
    
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")

    prompt_noreviews = create_prompt("")
    number_of_categories = len(categories)

    # Constants
    total_token_limit = total_token_limit
    estimated_max_output_per_review = sum([len(tokenizer.tokenize(s)) for s in review_categories(categories)])
    initial_prompt_tokens = len(tokenizer.tokenize(prompt_noreviews))
    reviews = preprocess_data(data)[0]

    # Set up loop
    batches = []
    current_batch = []
    current_token_count = initial_prompt_tokens

    for review in reviews:

        review_token_count = len(tokenizer.tokenize(review))

        current_token_count = current_token_count + review_token_count + estimated_max_output_per_review

        if current_token_count <= total_token_limit:
            current_batch.append(review)

        else:
            batches.append(current_batch)
            current_batch = [review]
            current_token_count = initial_prompt_tokens + review_token_count

    if current_batch:
        batches.append(current_batch)
    
    return batches, current_token_count

In [9]:
# Function to feed the data and review categories into the LLM

def get_ratings(batch=batch, categories=categories):
    
    '''Function to string together previous functions and produce a table with tailor-made
    review categories'''

    # Fill prompt
    categories_loop = review_categories(categories)
    prompt = create_prompt(result_string=batch, categories_loop=categories_loop)
    
    # Execute LLM prompting
    print(f'This is the prompt fed into the LLM:', prompt)
    ratings = get_completion(prompt)

    # Parse output (probably overcomplicating it)
    json_string = ratings.split('<result>')[1].split('</result>')[0]
    entries = json_string.split('"Id":')[1:]  
    
    data_list = []

    for entry in entries:
        formatted_entry = '{"Id":' + entry.strip().rstrip(',') + '},'
        formatted_entry = formatted_entry.replace(']}','}').replace('\n', ' ')
        formatted_entry = re.sub(' +', ' ', formatted_entry)
        data_list.append((formatted_entry))
    
    dict_list = [json.loads(item.rstrip(',')) for item in data_list]
    
    df = pd.DataFrame(dict_list)
    df = df.set_index('Id')

    pivot_table = df.pivot_table(index='Id',
                               columns='category', 
                               values=['assessment', 'score'], 
                               aggfunc='first')

    return pivot_table

In [10]:
# Function to execute LLM per batch, then append results to original data

def execute_script(data=data, prompt=prompt, categories=categories):
    
    print('Preprocess data')
    data_df = preprocess_data(data)[2]
    
    print('Create batches')
    batches = create_batches(data=data, categories=categories)
        
    print('Get ratings')
    outputs = [get_ratings(batch=batch, categories=categories) for batch in batches]
    combined_df = pd.concat(outputs)

    data_df.index = data_df.index.astype(int)
    combined_df.index = combined_df.index.astype(int)

    final_df = pd.concat([data_df,combined_df], axis=1)
    
    return final_df

In [11]:
def plot_output(final_df=final_df, categories=categories):
    number_of_categories = len(categories)
    df = final_df

    sentiment_columns = df.columns[-number_of_categories:]

    grouped_list = [df.groupby(['review_year', df[col]]).size().unstack().fillna(0) 
                    for col in sentiment_columns]
    grouped_list = [grouped.drop(columns=[0], errors='ignore') for grouped in grouped_list]

    merged_grouped = pd.concat(grouped_list, axis=1, keys=sentiment_columns)
    merged_grouped = merged_grouped.fillna(0)

    colors = plt.cm.viridis(np.linspace(0, 1, len(merged_grouped.columns)))

    fig, ax = plt.subplots(figsize=(6, 4))
    merged_grouped.plot(kind='bar', stacked=True, ax=ax, color=colors)
    ax.set_title(f"Stacked Bar Chart of Sentiments per Year")
    ax.set_ylabel("Count")
    ax.set_xlabel("Year")
    ax.legend(title="Score", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    plt.close(fig)
    
    return fig,ax

In [30]:
# Get data
file_list = glob.glob('*BOMA*.xlsx')
df_list = [pd.read_excel(file) for file in file_list]
data = pd.concat(df_list, ignore_index=True)

total_records = len(data)

# Initialize the extension
pn.extension()

# Define a function to calculate the cost based on the number of categories
def calculate_cost(data, categories):
    cost_per_token = 0.000002  # Cost input $0.0015 / 1K tokens; output $0.002 / 1K tokens
    estimated_tokens = len(create_batches(data, categories)[0])*4000+ create_batches(data, categories)[1]
    total_estimated_cost = estimated_tokens * cost_per_token
    return total_estimated_cost

# Define a function to process the review category and return some output
def process_category(event):
    categories = category_input.value.split(",")  # Split by comma
    # Strip potential spaces around the categories
    categories = [category.strip() for category in categories]
    cost = calculate_cost(data, categories)
    data_size = len(data)
    categories_size = len(categories)
    cost_pane.object = f"The estimated cost for processing {data_size} records for\
    {categories_size} categories is €{round(cost,3)}. Do you want to proceed?"
    confirm_button.visible = True

def confirm_execution(event):
    categories = category_input.value.split(",")  # Split by comma
    categories = [category.strip() for category in categories]
    # Convert the list of categories to a string representation and update the output pane
    result = f"You entered the categories: {', '.join(categories)}. \n\n Please be patient while the Analyst does her magic."
    output_pane.object = result
    
    analysis = execute_script(data=data[:5], categories=categories)
#     plot = plot_output(final_data=analysis, categories=categories)
    
    df_pane.object = analysis.loc[:, ['review_text'] + list(analysis.columns[-2*len(categories):])]

#     fig, ax = plot_output(final_df=analysis, categories=categories)
#     plot_pane.object = fig
        
# Create the widgets
category_input = pn.widgets.TextInput(name="Enter review categories, divided by comma's",
                                      placeholder='E.g. Friendliness of staff, quality of products, ...')
process_button = pn.widgets.Button(name="Calculate Cost", button_type="primary")
process_button.on_click(process_category)

confirm_button = pn.widgets.Button(name="Confirm & Execute", button_type="success", visible=False)
confirm_button.on_click(confirm_execution)

# Create an output pane to display the result
output_pane = pn.pane.Markdown("Output will be displayed here.")
cost_pane = pn.pane.Markdown()  # Display the calculated cost
df_pane = pn.pane.DataFrame()
plot_pane = pn.pane.Matplotlib(dpi=80)

# Create a layout for the app
layout = pn.Column(
    pn.pane.Markdown("## Micropole Google Maps Review Analyst"),
    pn.pane.Markdown("Analyse your reviews the way you want it!\nWe use fictional data for this example"),
    category_input,
    process_button,
    cost_pane,
    confirm_button,
    output_pane,
#     plot_pane,
    df_pane
)

# Display the app
layout.show()

Launching server at http://localhost:50891
